# Azure Cognitive Services で AI プログラミング

この演習では Azure Cognitive Services で提供されている **事前構築済みの AI** を利用して AI 開発を体験します。

学習モデル（AI の頭脳の当たるもの）を開発するには機械学習やプログラミングの知識が必要ですが、事前構築済みの学習モデルを利用すると非常に簡単に AI プログラミングができます。  
ここでは AI で何ができるかを自分で実際に操作して体験してみます。

> "Azure Cognitive Services で AI プログラミング" を実施するには [**Azure サブスクリプション**](https://azure.microsoft.com/ja-jp/) が必要です。  
> アカウントを持っていない場合は [**無料の評価版アカウントを取得**](https://azure.microsoft.com/ja-jp/free/) することができます。

---

## 0. 準備 - Cognitive Services 接続情報を確認

最初に Cognitive Services への接続情報が正しく設定されているかを確認します。

接続情報が設定されていれば、以下のセルを実行するとエンドポイントとキーの値が表示されます。

- エンドポイント・・・ "https://" で始まる文字列
- キー・・・英数字がランダムに並んだ文字列

接続情報がまだ設定されていない場合は、この Notebook を実行する前に以下の手順を先に実施してください。

1. [**Cognitive リソースの作成**](./a01_createcog.ipynb)
1. [**演習の環境構築**](./0_setup.ipynb)

> 集合形式の演習ではあらかじめ接続情報が設定されているかもしれません。その場合は講師やスタッフから指示や連絡がありますので、それに従ってください。

In [ ]:
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()
cog_endpoint = os.getenv('COG_SERVICE_ENDPOINT')
cog_key = os.getenv('COG_SERVICE_KEY')

In [ ]:
print("Endpoint: " + cog_endpoint)
print("Key: " + cog_key)

---

## 1. 画像分析

AI の活用例としてよく取り上げられるのが **画像分析** です。

画像分析は、

- 画像に概要説明を付ける
- 画像の特徴となるキーワードを答える
- 人の顔の位置を答える
- 人の年齢、性別などを推測する

などを行うことです。

画像データは AI で扱いやすく結果を目で見てわかることから、具体的に理解しやすい題材です。

> 画像データは高解像度の写真だとしても事前にデータ量が分かります。"機械的に処理しやすい" (= 決まった手順で処理しやすい) のでコンピューターで扱いやすいデータです。

---

最初に、Cognitive Services の **Computer Vision** (画像処理を行う事前構築済みの学習モデル) を利用するためのパッケージを読み込みます。  
一緒に Notebook 上に画像を表示するためのパッケージも読み込みます。

パッケージとは、それぞれある機能を実現するために "**いくつかの関数**" をまとめたものです。  
Python で利用できるパッケージは [**PyPI**](https://pypi.org/) に多数登録されていて、これらをインストールすることで毎回面倒なコードを書かなくても必要な機能を利用できるものです。

In [ ]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

---

解析する画像ファイルを読み込んで、どのような画像なのか表示して確認してみます。

> 余裕があれば、下のセルで image1.jpg を image2.jpg に変更して改めて実行してみてください。

In [ ]:
image_path = os.path.join('data', 'image', 'image1.jpg')

# 余裕があればこちらのように "image2.jpg" についても実行
# image_path = os.path.join('data', 'image', 'image2.jpg')
# image_path = os.path.join('data', 'image', 'ota_seo.jpg')
# image_path = os.path.join('data', 'image', 'omori_seo.jpg')

In [ ]:
img = Image.open(image_path)
plt.axis('off')
plt.imshow(img)

---

Computer Vision に接続します。

In [ ]:
# Get a client for the computer vision service
computervision_client = ComputerVisionClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

---

上の画像を Computer Vision に渡して分析します。

In [ ]:
image_stream = open(image_path, "rb")

features = ['Description', 'Tags', 'Faces']
analysis = computervision_client.analyze_image_in_stream(image_stream, visual_features=features, language="ja")

---

画像の説明を見てみます。  
説明の **確からしさ** (=予測にどのくらい自信があるか) も合わせて表示します。

In [ ]:
for caption in analysis.description.captions:
    print(f'{caption.text} ({(caption.confidence * 100):.1f}%)')

---

画像に写っているものを表示します。  
それぞれの "**確からしさ**" も合わせて表示します。

In [ ]:
for tag in analysis.tags:
    print(f'{tag.name} ({(tag.confidence * 100):.1f}%)')

---

画像に写っている「人」について、以下の情報を表示します。

- 顔が写っている人の人数
- それぞれの人の顔の位置（赤枠で囲みます）
- それぞれの人の年齢
- それぞれの人の性別

年齢、性別については予測なので正解とは限りません。（人間でも年齢、性別を正確に当てられないのと同じです）

In [ ]:
import matplotlib.patches as patches

plt.axis('off')
plt.imshow(img)

ax = plt.gca()

for face in analysis.faces:
    bound = face.face_rectangle
    rect = patches.Rectangle((bound.left, bound.top), bound.width, bound.height, linewidth=4, edgecolor='red', fill = False)
    ax.add_patch(rect)

plt.show()

print(f'{len(analysis.faces)}人の人が写っています')

for face in analysis.faces:
    print(f'{"男性" if face.gender == "Male" else "女性"} {face.age}歳')

> 上では if - else を 1行で記述しています。
>
> ```python
> 条件が成立する時の値 if 条件式 else 条件が成立しない時の値  
> ```  
>
> これを三項演算子といいます。

---

## 2. 自然言語処理

自然言語処理とは、人間の **話し言葉、書き言葉** を理解する能力のことです。

画像処理に比べるとコンピューターで処理するのは難しいデータです。  
データが終わるまでサイズ（文の長さ）がわからないなどの特徴があるため処理方法が複雑になりやすいからです。

また特に日本語は、

- 単語の区切りの位置がわかりづらい（英語だと単語ごとにスペースが入るので切れ目がわかる）
- 語順の入れ替えが起こりやすい（「私はコンピューターで自然言語処理を行います」と「私は自然言語処理をコンピューターで行います」は同じ意味）
- 文末まで意味が確定しないことがある（「私はそう思いま」に続くのが「す」か「せん」かで意味が逆転する）

などのように自然言語処理でも特に難しい言語です。

> 日本語の処理が難しいのはこの後の演習で実際に確認できます。

---

Cognitive Services の **Text Analytics** (自然言語処理を行う事前構築済みの学習モデル) を利用するためのパッケージを読み込みます。

In [ ]:
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

演習で使用する文章を表示して確認してみます。

In [ ]:
review_folder = os.path.join('data', 'text')

reviews = []
for filename in os.listdir(review_folder):
    if filename.endswith(".txt"):
        text = open(os.path.join(review_folder, filename), 'r', encoding='UTF-8').read()
        review = {"id": filename, "text": text}
        reviews.append(review)

In [ ]:
for review in reviews:
    print(f'--- {review["id"]} ---\n{review["text"]}\n')

> 英語の文章が一つ、日本語の文章が二つ表示されるはずです。  
> 英語の文章はポジティブ（好意的）な内容、日本語の文章はポジティブな内容のものとネガティブな内容のものであることを確認してください。

---

Text Analytics に接続します。

In [ ]:
text_analytics_client = TextAnalyticsClient(cog_endpoint, CognitiveServicesCredentials(cog_key))

上の文章を Text Analytics に渡します。

In [ ]:
language_analysis = text_analytics_client.detect_language(documents=reviews)

---

それぞれの文章が何語で書かれているか予測します。

In [ ]:
for review_num in range(len(reviews)):
    lang = language_analysis.documents[review_num].detected_languages[0]
    print(f'{reviews[review_num]["id"]}: {lang.name} ({(lang.score * 100):.1f}%)')
    
    reviews[review_num] = {"id": reviews[review_num]["id"], "text": reviews[review_num]["text"], "language": lang.iso6391_name}

> 下の処理でそれぞれの文章のキーフレーズを抽出します。  
> これについては何語で書かれているかを渡すほうが正確に処理できます。
>
> そこで
>
> ```python
>  reviews[review_num] = {"id": reviews[review_num]["id"], "text": reviews[review_num]["text"], "language": lang.iso6391_name}
> ```
>
> で、予測した言語を reviews リストの各要素に追加しています。("language": ～ の部分)

---

それぞれの文章のキーフレーズ（主要な単語やフレーズ）を取得します。

In [ ]:
key_phrase_analysis = text_analytics_client.key_phrases(documents=reviews)

In [ ]:
for review_num in range(len(reviews)):
    print(f'{reviews[review_num]["id"]} のキーフレーズ:')

    key_phrases = key_phrase_analysis.documents[review_num].key_phrases
    # Print each key phrase
    for key_phrase in key_phrases:
        print('\t', key_phrase)
        
    print()

---

それぞれの文章の感情分析を行います。

スコアが 50% (= 0.5) がニュートラル（中間）です。  
100% (= 1.0) に近いほどポジティブな文章、0% (= 0.0) に近いほどネガティブな文章であることを意味します。

In [ ]:
sentiment_analysis = text_analytics_client.sentiment(documents=reviews)

In [ ]:
for review_num in range(len(reviews)):
    sentiment_score = sentiment_analysis.documents[review_num].score
    sentiment = "ネガティブ" if sentiment_score < 0.5 else "ポジティブ"
    print(f'{sentiment_analysis.documents[review_num].id}: {sentiment} ({(sentiment_score * 100):.1f}%)')

> 日本語の処理は難しいため、英語に比べてポジティブ、ネガティブとも 50% に近い値になる傾向があります。

---

一般に AI 開発には数学や統計学の知識とプログラミングのスキルが必要です。  
しかし事前構築済みの AI である Cognitive Services を利用すると、画像や自然言語を分析することが比較的簡単にできることが分かったと思います。  

解きたい問題（＝コンピューターにやらせたいこと）によって、事前構築済みの学習済みモデルを使うほうがよいものと開発者が学習モデルを作るほうがよいものとがあります。  
それらの使い分けや学習モデルの開発方法などは今回は扱いませんが、まずは AI で何ができるのかを体験できたと思います。

興味がある方は Cognitive Services のドキュメントや AI の基礎の勉強用サイトを活用してみてください。

- [Cognitive Services のドキュメント](https://docs.microsoft.com/ja-jp/learn/browse/?products=azure-cognitive-services)
- [AI の基礎 (認定資格 AI-900 の勉強用サイト)](https://docs.microsoft.com/ja-jp/users/seosoft/collections/3goyt00qjmxk7m)

---

Cognitive Services リソースを自分で作成した場合は、演習終了後に削除してください。  
削除の手順は [こちら](102_deletecog.ipynb) です。

集合演習などであらかじめ Cognitive Services リソースが用意されていた場合は、特に何もする必要はありません。